In [97]:
import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt
import shap
import sys
import torch

In [98]:
sys.path.append('..')

In [99]:
from convlstm_classes import EncoderDecoderConvLSTM

In [100]:
dataset = np.load('../../convlstm/splits/dataset_normalized.npy')

In [101]:
X = dataset[:,:11]

In [102]:
y = dataset[:,11:,1]

In [103]:
len_full = len(X)

In [104]:
len_test = int(0.2 * len_full)

In [105]:
len_test

162

In [106]:
X_test = X[-len_test:]

In [107]:
np.random.seed(42)

In [108]:
test_sample_indices = np.random.choice(range(len_test), size=40, replace=False)

In [139]:
X_sample = X_test[test_sample_indices]

In [140]:
X_sample = X_sample.astype('float32')

In [141]:
model = EncoderDecoderConvLSTM()

In [142]:
model.load_state_dict(torch.load('../best_model_params.pt', map_location='cpu'))

<All keys matched successfully>

In [143]:
model.eval()

EncoderDecoderConvLSTM(
  (encoder_1_convlstm): ConvLSTMCell(
    (conv): Conv2d(32, 96, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  )
  (encoder_2_convlstm): ConvLSTMCell(
    (conv): Conv2d(48, 96, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  )
  (batchnorm1): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (decoder_1_convlstm): ConvLSTMCell(
    (conv): Conv2d(48, 96, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  )
  (decoder_2_convlstm): ConvLSTMCell(
    (conv): Conv2d(48, 96, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  )
  (batchnorm2): BatchNorm3d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (decoder_CNN): Conv3d(24, 1, kernel_size=(1, 3, 3), stride=(1, 1, 1), padding=(0, 1, 1))
)

In [146]:
def predict(x):
    with torch.no_grad():
        return np.array(model(torch.tensor(x)).squeeze(2)[:,1,5,10])

In [147]:
explainer = shap.KernelExplainer(model=predict, data=X_sample)

In [148]:
shap_values = explainer.shap_values(X_sample)

DimensionError: Instance must have 1 or 2 dimensions!